In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import CharacterTextSplitter

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

embeddings = HuggingFaceEmbeddings(model_name=model_name)
print("Model muvaffaqiyatli yuklandi!")

/Users/uzbtrust/Desktop/Hallucination Detector/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Model muvaffaqiyatli yuklandi!


In [2]:
text_data = """
Mohamed bin Zayed sun'iy intellekt universiteti (MBZUAI) Birlashgan Arab Amirliklarining 
Abu-Dabi shahrida joylashgan dunyodagi birinchi sun'iy intellektga ixtisoslashgan oliygohdir. 
Universitet 2019-yilda tashkil etilgan.
"""

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = text_splitter.create_documents([text_data])

vector_db = FAISS.from_documents(docs, embeddings)
print("Vektor bazasi tayyor!")

Vektor bazasi tayyor!


In [3]:
query = "MBZUAI qayerda joylashgan?"
results = vector_db.similarity_search(query)

print(f"Savol: {query}")
print(f"Topilgan javob: {results[0].page_content}")

Savol: MBZUAI qayerda joylashgan?
Topilgan javob: Mohamed bin Zayed sun'iy intellekt universiteti (MBZUAI) Birlashgan Arab Amirliklarining 
Abu-Dabi shahrida joylashgan dunyodagi birinchi sun'iy intellektga ixtisoslashgan oliygohdir. 
Universitet 2019-yilda tashkil etilgan.


In [4]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Meta-Llama-3-8B-Instruct",
    huggingface_api_token=hf_token,
    temperature=0.1
)

template = """Siz MBZUAI yordamchisiz. Faqat kontekstga asoslanib javob bering.
Kontekst: {context}
Savol: {question}
Javob:"""
prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = vector_db.as_retriever()

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("RAG tizimi LCEL yordamida muvaffaqiyatli qurildi!")

WARNING! huggingface_api_token is not default parameter.
                    huggingface_api_token was transferred to model_kwargs.
                    Please make sure that huggingface_api_token is what you intended.


RAG tizimi LCEL yordamida muvaffaqiyatli qurildi!
